<a href="https://colab.research.google.com/github/ArjunDandagi/cs145/blob/main/cs_145_Hashing_Algos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybloom_live h3 faiss-cpu whoosh

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.6/993.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.9 MB/s eta 0:00:00
  Created wheel for pybloom_live: filename=pybloom_live-4.0.0-py3-none-any.whl size=9228 sha256=e13b129a7cd7bd35525e0658658dd6b871eff340c2f6378bb6703c52077fa746
  Stored in directory: /root/.cache/pip/wheels/36/8a/9f/83ab00e9a9b2b10cec6135fa2a6cd92a22edf3d87fdaae481e
Successfully built pybloom_live


In [ ]:
# Basic hash function
import hashlib
def hash_value(value, r=8):
  if isinstance(value, int):
    return hash(value) % r
  sha256 = hashlib.sha256()
  sha256.update(value.encode('utf-8'))
  return int(sha256.hexdigest(), 16)

In [ ]:
print(f'{hash_value("String 42 is making me hungry")=}')
print(f'{hash_value("String 43 is making me hungry")=}')

hash_value("String 42 is making me hungry")=115370750186445326290377683743957786138830981293626300636188810120500006517549
hash_value("String 43 is making me hungry")=43094278518137645842364210990503851658090982228390781508461084004583964858452


In [ ]:
print(f'{hash_value(421, 8)=}')
print(f'{hash_value(1523789, 8)=}')
print(f'{hash_value("Crazy")=}')
print(f'{hash_value("Crazy")=}')
print(f'{hash_value("Taylor Swift")=}')
print(f'{hash_value("(37.428226, -122.174722)")=}') # Stanford NVidia

print(f'{hash_value("String 42 is making me hungry")=}')
print(f'{hash_value("String 43 is making me hungry")=}')
print(f'{hash_value("(37.428226, -122.174722)")=}') # Stanford NVidia
print(f'{hash_value("(37.429749, -122.173549)")=}') # Stanford Gates
print(f'{hash_value("(37.428226, -122.174722)")=}') # Stanford NVidia




hash_value(421, 8)=5
hash_value(1523789, 8)=5
hash_value("Crazy")=26532752625765826853685626387507575286736774457388345789099730774055940208863
hash_value("Crazy")=26532752625765826853685626387507575286736774457388345789099730774055940208863
hash_value("Taylor Swift")=88112730712611275922716045032483444911241879675316572061569441555880719989902
hash_value("(37.428226, -122.174722)")=15584662638934651596674661938880452430485871690911786775291271296878253906639
hash_value("String 42 is making me hungry")=115370750186445326290377683743957786138830981293626300636188810120500006517549
hash_value("String 43 is making me hungry")=43094278518137645842364210990503851658090982228390781508461084004583964858452
hash_value("(37.428226, -122.174722)")=15584662638934651596674661938880452430485871690911786775291271296878253906639
hash_value("(37.429749, -122.173549)")=96461388757156338915254904484651934017607226452294372348554236985010151672039
hash_value("(37.428226, -122.174722)")=155846626389346515

In [ ]:
import os
from whoosh.fields import TEXT, Schema
from whoosh.index import create_in
from whoosh.qparser import QueryParser

# Define the index schema
schema = Schema(content=TEXT)
os.mkdir("my_index_dir2")
# Create a new index
ix = create_in("my_index_dir2", schema)

# Add some documents to the index
with ix.writer() as writer:
  writer.add_document(content="crazy in love - beyonce")
  writer.add_document(content="crazy love - van morrison")
  writer.add_document(content="crazy -- aerosmith")

# Search for documents containing the word "fox"
qries = ["crazy", "aerosmith", "crazy love"]
for q in qries:
  with ix.searcher() as searcher:
    results = searcher.search(QueryParser("content", schema).parse(q))
    print(results)


<Top 3 Results for Term('content', 'crazy') runtime=0.0004599319999982754>
<Top 1 Results for Term('content', 'aerosmith') runtime=0.00039411400000233243>
<Top 2 Results for And([Term('content', 'crazy'), Term('content', 'love')]) runtime=0.0008488379999960216>


In [ ]:
import h3
# lat/lngs from SF mission, Stanford NVidia, Packard, Gates Buildings
locations = [(37.788022, -122.399797), (37.428226, -122.174722),
             (37.429749, -122.1735490), (37.429761, -122.173290)]
for zoom in [8, 12]: # zoomlevels 8 and 12 (imagine the +/- in map zoom)
  for l in locations:
    cell_id = h3.geo_to_h3(l[0], l[1], zoom)
    print("@zoom[", zoom, "]:", cell_id)

AttributeError: module 'h3' has no attribute 'geo_to_h3'

In [ ]:
from pybloom_live import BloomFilter

# Create a Bloom filter with 1000 items and a 0.1% false positive rate
bf = BloomFilter(capacity=1000, error_rate=0.001)

# Add some integers to the Bloom filter
bf.add(42); bf.add(30); bf.add(50)

# Check if each integer in the list is in the Bloom filter
integer_list = [42, 50, 100, 32]
for i in integer_list:
  if i in bf:
    print(f"{i} may be in the Bloom filter")
  else:
    print(f"{i} is definitely not in the Bloom filter")

In [ ]:
import numpy as np, faiss
# Generate 1000 points in a 5000-dimensional space
X = np.random.rand(10000, 5000).astype('float32')

# Build an LSH index for the points
index = faiss.IndexLSH(5000, 16)
index.add(X)

# Find the 5 nearest neighbors (distances and points) of xq, a new 10-dim point
xq = np.random.rand(5000).astype('float32')
D, I = index.search(np.expand_dims(xq, axis=0), k=5)

# Print the indices and distances of the nearest neighbors
print("Nearest neighbors of ", xq, " are:", I, " with distance: ", D)

In [ ]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-distilroberta-base-v1')

text = "String 42 is making me thirsty."
embedding = model.encode(text)

print(embedding)

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# Create 1000 example strings
strings = [f"String {i} is making me hungry" for i in range(1000)]

# Load the SentenceTransformer model
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

# Compute embeddings for the strings
embeddings = model.encode(strings)

# Create a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Add the embeddings to the index
index.add(np.array(embeddings))

# Define a query string and compute its embedding
for query_string in ["String 42 is making me hungry",
                     "String 42", "String 42 is making me thirsty"]:
  query_embedding = model.encode(query_string)

  # Search the index for the nearest neighbors of the query embedding
  k = 5  # Number of nearest neighbors to retrieve
  distances, indices = index.search(np.array([query_embedding]), k)

  print(f'{query_string=} ==> ')
  # Print the results
  for i, idx in enumerate(indices[0]):
    print(f"Rank {i+1}: {strings[idx]}, Distance: {distances[0][i]}")


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Create a sample dataframe
df = pd.DataFrame({
    'col1': [1, 2, 3, 4],
    'col2': ['foo', 'bar', 'baz', 'qux'],
    'col3': [0.1, 0.2, 0.3, 0.4],
})

# Convert the dataframe to an Arrow table
table = pa.Table.from_pandas(df)

# Write the table to a Parquet file
pq.write_table(table, 'example.parquet', compression='snappy')

# Read the Parquet file back into an Arrow table
table_from_file = pq.read_table('example.parquet')

# Convert the Arrow table back to a Pandas dataframe
df_from_file = table_from_file.to_pandas()

# Print the original and read dataframes to verify they match
print(df)
print(df_from_file)